# MT Metadata Example 5

## Experiment

We have covered some basics of MT metadata, lets put it together into an `Experiment`.

In [1]:
from mt_metadata.timeseries import (
    Auxiliary,
    Electric,
    Experiment,
    Magnetic,
    Run,
    Station,
    Survey,
)
from mt_metadata.timeseries.filters import CoefficientFilter

In [2]:
experiment = Experiment()

# set a common start and end time
start = "2020-01-01T00:00:00+00:00"
end = "2021-01-01T12:00:00+00:00"

kwargs = {"time_period.start": start, "time_period.end": end}
cf1 = CoefficientFilter(name="filter_01", gain=10)
cf2 = CoefficientFilter(name="filter_02", gain=100)

ch1_kwargs = {
    "time_period.start": start,
    "time_period.end": end,
    "filter.name": ["filter_01"],
    "comments": "run_ids: mt01a, mt01b",
}
ch2_kwargs = {
    "time_period.start": start,
    "time_period.end": end,
    "filter.name": ["filter_02"],
    "comments": "run_ids: mt02a, mt02b",
}

for survey in ["One", "Two"]:
    survey_obj = Survey(id=survey)
    survey_obj.fdsn.network = "EM"
    survey_obj.acquired_by.author = "M T"
    survey_obj.project_lead.author = "T M"
    if survey in ["One"]:
        survey_obj.filters[cf1.name] = cf1
    else:
        survey_obj.filters[cf2.name] = cf2
    for station in ["mt01", "mt02"]:
        station_obj = Station(id=station, **kwargs)
        for run in ["mt01a", "mt01b"]:
            run_obj = Run(id=run, **kwargs)
            if survey in ["One"]:
                for ch in ["ex", "ey"]:
                    ch_obj = Electric(component=ch, **ch1_kwargs)
                    ch_obj.dipole_length = 100
                    run_obj.channels.append(ch_obj)
                for ch in ["hx", "hy", "hz"]:
                    ch_obj = Magnetic(component=ch, **ch1_kwargs)
                    run_obj.channels.append(ch_obj)
                for ch in ["temperature"]:
                    ch_obj = Auxiliary(component="tx", **ch1_kwargs)
                    ch_obj.type = ch
                    ch_obj.measured_azimuth = 0
                    run_obj.channels.append(ch_obj)
            else:
                for ch in ["ex", "ey"]:
                    ch_obj = Electric(component=ch, **ch2_kwargs)
                    ch_obj.dipole_length = 100
                    run_obj.channels.append(ch_obj)
                for ch in ["hx", "hy", "hz"]:
                    ch_obj = Magnetic(component=ch, **ch2_kwargs)
                    run_obj.channels.append(ch_obj)
                for ch in ["temperature"]:
                    ch_obj = Auxiliary(component="ty", **ch2_kwargs)
                    ch_obj.type = ch
                    ch_obj.measured_azimuth = 90
                    run_obj.channels.append(ch_obj)
            run_obj.update_time_period()
            station_obj.runs.append(run_obj)
            station_obj.update_time_period()
        survey_obj.stations.append(station_obj)
        survey_obj.update_time_period()
    experiment.surveys.append(survey_obj)

In [3]:
experiment

Experiment Contents
--------------------
Number of Surveys: 2
  Survey ID: One
  Number of Stations: 2
  Number of Filters: 1
  --------------------
    Filter Name: filter_01
    Filter Type: coefficient
    --------------------
    Station ID: mt01
    Number of Runs: 2
    --------------------
      Run ID: mt01a
      Number of Channels: 6
      Recorded Channels: ex, ey, hx, hy, hz, tx
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
      Run ID: mt01b
      Number of Channels: 6
      Recorded Channels: ex, ey, hx, hy, hz, tx
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
    Station ID: mt02
    Number of Runs: 2
    --------------------
      Run ID: mt01a
      Number of Channels: 6
      Recorded Channels: ex, ey, hx, hy, hz, tx
      Start: 2020-01-01T00:00:00+00:00
      End:   2021-01-01T12:00:00+00:00
      --------------------
      Run ID: mt01b
      Numbe

### To JSON

We can write this to a JSON file, these can get long, but there are readers out there that make it easy to search.

In [4]:
experiment.to_json(fn="test.json")

### To XML
We can write an XML file if you really want to.

In [9]:
xml_element = experiment.to_xml(fn="test.xml")

### StationXML

FDSN uses StationXML to archive which can sometimes be a pain to translate to.  `mt_metadata` provides translation tools to and from StationXML.  The StationXML format that is output has been extensively adjusted to fit both MT and FDSN standards, so if possible these tools should be used to create StationXML metadata files from MT Experiments.

For the StationXML side `Obspy` is used to be sure the format is correct.  

Before we do the conversion we need to be sure that some important metadata are included in experiment.

The translations are in `mt_metadata.timeseries.stationxml`

In [6]:
from mt_metadata.timeseries.stationxml import xml_inventory_mt_experiment

translator = xml_inventory_mt_experiment.XMLInventoryMTExperiment()

In [7]:
inventory = translator.mt_to_xml(experiment, stationxml_fn="test_stationxml.xml")

The created StationXML can be the StationXML you use for archiving.